# Here I will simply build a classifier that detects the presence of a red ball or not in the training imagaes.
    - First build classifier
    - Then try to do the sliding window thing.
    - I think the purpose of sliding window is so we know the new location of the ball. E.g. If we find ball in top right then move in that direction.

In [21]:
# imports
import os
import numpy as np
from PIL import Image   # https://www.pluralsight.com/guides/importing-image-data-into-numpy-arrays
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from scipy import stats

In [22]:
# Define dirs
pos_img_dir = 'TrainingImages/positives/'
neg_img_dir = 'TrainingImages/negatives/'

# go through negatives
neg_images = []
neg_labels = np.zeros( len( os.listdir(neg_img_dir) ) )

for filename in os.listdir(neg_img_dir):
    neg_images.append( np.array( Image.open( neg_img_dir + filename ).convert('L') )/255 )

print(np.shape(neg_images))

# go through positives
pos_images = []
pos_labels = np.ones( len( os.listdir(pos_img_dir) ) )

for filename in os.listdir(pos_img_dir):
    pos_images.append( np.array( Image.open( pos_img_dir + filename ).convert('L') )/255 )

print(np.shape(pos_images))

(2600, 51, 51)
(308, 51, 51)


In [23]:
# Now weve got data in array form we can split into test and train data?
# With GREYSCALE images
# TODO Try with full colour images

X = np.concatenate((neg_images, pos_images))
y = np.concatenate((neg_label, pos_label))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=300, shuffle=True)
#Prove test and train data is split correctly
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
X_train[0].shape

NameError: name 'neg_label' is not defined

In [0]:
# Try flattening for sklearn to work
fX_train = np.reshape(X_train, (len(y_train),-1))
print(fX_train.shape)
fX_test = np.reshape(X_test, (len(y_test),-1))
print(fX_test.shape)

In [0]:
# Try a Random forrest classifier on image?
# Using fX, flattened 1D array for each image
# TODO still might have to change so I actually extract features
# TODO perhaps use a CNN with all colours first?

clf=RandomForestClassifier(n_estimators=50)
clf.fit(fX_train,y_train)
y_pred=clf.predict(fX_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
import PacMan_functionality as PacMan

# Initialise game
global_cloud, spheres_collected = PacMan.startup_scene()

In [0]:

# Create current camera position and angle
position = np.zeros([3])
angle = np.zeros([3])

# Obtain image and associated maps
image, mapx, mapy, mapz, depth = PacMan.project_pointcloud_image(global_cloud, angle, position)
type(depth)

In [0]:
# Plot images using matplotlib.pyplot
plt.figure(figsize=(4,2))
plt.imshow(image)
plt.colorbar()

plt.figure(figsize=(4,2))
plt.imshow(mapx)
plt.colorbar()

plt.figure(figsize=(4,2))
plt.imshow(mapy)
plt.colorbar()

plt.figure(figsize=(4,2))
plt.imshow(mapz)
plt.colorbar()

plt.figure(figsize=(4,2))
plt.imshow(depth, cmap='bwr')
plt.colorbar()

In [0]:
# Really stupid but here goes
# To convert the image into greyscale the same way I did when training the data I need to save the image as a png then reopen it
# TODO fix in the future if i can but cant atm
plt.imsave('image.png', image)
greyimg = Image.open('image.png').convert('L')
greyimgnp = np.asarray(greyimg)
print(greyimgnp.shape)
greyimg

In [0]:
def sliding_window_classifier( step_size, image ):
    #image is (160, 240, 3)
    #window is (51, 51, 3)
    image_hight = image.shape[0] # 160
    image_width = image.shape[1] # 240
    window_size = 51
    windows = []

    for y in range(0, image_hight-window_size, step_size): 
        for x in range (0, image_width-window_size, step_size):
            test = image[y:y+window_size, x:x+window_size] # Had to remove last : because grey image has no colurs

            prediction = clf.predict(test.reshape(1,-1))

            if (prediction==1):
                print(i)
                print('X:{}, Y:{}'.format(x,y))
                print(prediction)
                plt.figure(figsize=(4,2))
                plt.imshow(test, cmap='gray', vmin=0, vmax=255)    
                plt.show()

print(greyimgnp.shape)
sliding_window_classifier(20, greyimgnp)

depthim = depth[60:60+51, 60:60+51]
plt.figure(figsize=(4,2))
plt.imshow(depth[60:60+51, 60:60+51], cmap='bwr')
plt.colorbar()


In [0]:
flatdepth = depthim.reshape(-1,1)               # From 2d array to 1d array
flatdepth = np.round(flatdepth, decimals=1)     # Round array to 1 decimal place
mode, count = stats.mode(flatdepth)             # Calculate mode of array
# Mode will be the most frequent item in the array
# Rationale is that the most frequent item will be the sphere, as the sphere takes up most of the screen?
# TODO Use different scales with the sliding window, you may just want to go forward if you can see one only a little bit?
mode[0,0] # Should be how far away the image is from us

In [0]:
spheres_collected

In [0]:
# Move camera 2.3 forwards, towards sphere
import PacMan_functionality as PacMan
import numpy as np

# Initilise game
global_cloud, spheres_collected = PacMan.startup_scene()
position = np.asarray([0, 0, 0])
print('I am at position [{0}, {1}, {2}]'.format(*position))
print('I have found {0} out of {1} spheres!\n'.format(np.sum(spheres_collected), len(spheres_collected)));

# Move position and update scene
#position = np.asarray([-0.18, 0.08, 2.31])
position = np.asarray([0, 0, 2])
print('I am now at position [{0}, {1}, {2}]'.format(*position))
global_cloud, spheres_collected = PacMan.update_scene(position, spheres_collected);
print('I have found {0} out of {1} spheres!\n'.format(np.sum(spheres_collected), len(spheres_collected)));

In [0]:
# Really stupid but here goes
# To convert the image into greyscale the same way I did when training the data I need to save the image as a png then reopen it
# TODO fix in the future if i can but cant atm
plt.imsave('image.png', image)
greyimg = Image.open('image.png').convert('L')
greyimgnp = np.asarray(greyimg)
print(greyimgnp.shape)
greyimg

In [0]:
subimage = greyimgnp[60:60+51, 70:70+51]
plt.imshow(greyimgnp[60:60+51, 70:70+51], cmap='gray', vmin=0, vmax=255)    # get a 51 by 51 size chunk of the image
plt.show()
print(clf.predict(subimage.reshape(1,-1)))  # It finds the point!!
clf.predict_proba(subimage.reshape(1,-1))

In [0]:
# Now ive proven my algorithm can find the point lets write some sliding window thing to break up the image into n smaller ones
print(greyimgnp.shape)

y=0
start_x = 0
start_y = 0
window_size = 51
move_size = 20
count=0
while(start_x<160):
    print()
    print('Count: ', count)
    #print('x:{}, y:{}'.format(start_x, start_y))
    end_x = start_x + window_size
    end_y = start_y + window_size

    curr_window = greyimgnp[start_x:end_x, start_y:end_y]
    #print(curr_window)
    #plt.imshow(curr_window, cmap='gray', vmin=0, vmax=255)
    #plt.show()
    
    try:        
        predict = clf.predict(curr_window.reshape(1,-1)) 
        if (predict==[1.]):
            print('FOUND SPHERE')
            print(start_x, start_y)
            #break
    except:
        print('WRONG SHAPE')

    start_y = start_y + move_size
    if (start_y + window_size > 240):
        # Window has gone off y
        # Restart at y=0 and increase x
        start_y = 0
        start_x = start_x + move_size

    start_y = start_y + move_size
    count = count+1

In [0]:
# Better sliding window
# Define function sliding window
# returns array of windows that I can classify at once
def sliding_window( step_size, image ):
    #image is (160, 240, 3)
    #window is (51, 51, 3)
    image_hight = image.shape[0] # 160
    image_width = image.shape[1] # 240
    window_size = 51
    windows = []

    for y in range(0, image_hight-window_size, step_size): 
        for x in range (0, image_width-window_size, step_size):
            windows.append(image[y:y+window_size, x:x+window_size]) # Had to remove last : because grey image has no colurs

    return np.array(windows) # Return windows as np array

res = sliding_window(20, greyimgnp)
np.shape(res)


In [0]:
test = sliding_window(20, greyimgnp) 
print(np.shape(test))
for i in range( len(test[:,0,0]) ):
    
    #prediction = model.predict( np.array( [test[i,:,:,:],]) )

    prediction = clf.predict(test[i,:,:].reshape(1,-1))

    if (prediction==0):
        continue
    else:
        print(i)
        print(prediction) 
        print()
        #print( str(prediction).format(1.0e-9) ) # Force scientific notation off
        plt.figure(figsize=(4,2))
        plt.imshow(test[i,:,:], cmap='gray', vmin=0, vmax=255)    
        plt.show()

In [0]:
#

In [0]:
#